In [2]:
import torch
import os
from scipy.cluster.vq import kmeans2
from sklearn.cluster import kmeans_plusplus
from sklearn.cluster import SpectralClustering
from matplotlib import pyplot as plt
import torch.nn.functional as F

In [3]:
#scop_embeddings_path = '../data/SCOPe40.pt'
scop_embeddings_path = '/scicore/home/schwede/pantol0000/repositories/alphabeta_benchmark/SCOPe40_label_transfer/data/embeddings.pt'
embeddings_dict = torch.load(scop_embeddings_path, map_location=torch.device('cpu'))
print(f"Landscape of {len(embeddings_dict)} proteins")

/scratch/ipykernel_221356/3220096321.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings_dict = torch.load(scop_embeddings_path, map_location=torch.device('cpu'))

Landscape of 5305 proteins


In [4]:
reduced_embeddings_dict = dict()
n_proteins = 500
for k in embeddings_dict:
    reduced_embeddings_dict[k] = embeddings_dict[k]
    n_proteins-=1
    if n_proteins<0:
        break

### Embedding normalization

In [5]:
standard_embedding_single = dict()
standard_embedding_land = dict()

sum_embeddings = torch.zeros(1024)
sum_squares_embeddings = torch.zeros(1024)
total_count = 0

for p in reduced_embeddings_dict:
    channel_sum = reduced_embeddings_dict[p].sum(dim=0)
    channel_squared_sum = (reduced_embeddings_dict[p] ** 2).sum(dim=0)
    length = reduced_embeddings_dict[p].size(0)

    sum_embeddings += channel_sum
    sum_squares_embeddings += channel_squared_sum
    total_count += length

    single_mean = channel_sum/length
    single_std = torch.sqrt( (channel_squared_sum/length) - (single_mean ** 2))
    
    standard_embedding_single[p] = (reduced_embeddings_dict[p] - single_mean)/single_std

mean_per_channel = sum_embeddings / total_count
print(f'Average {mean_per_channel}')
# Compute the variance for each channel
variance_per_channel = (sum_squares_embeddings / total_count) - (mean_per_channel ** 2)
# Compute the standard deviation for each channel
std_per_channel = torch.sqrt(variance_per_channel)
print(f'Standard deviation {std_per_channel}')

Average tensor([ 0.0306,  0.0228,  0.0157,  ..., -0.0093,  0.0221,  0.0134])
Standard deviation tensor([0.2043, 0.1741, 0.2012,  ..., 0.1849, 0.1960, 0.2010])


In [6]:
for p in reduced_embeddings_dict:
    standard_embedding_land[p] = (reduced_embeddings_dict[p] - mean_per_channel)/std_per_channel

In [7]:
del embeddings_dict

In [ ]:
tensor_list = [x for x in standard_embedding_single.values()]
data_landscape = torch.cat(tensor_list, dim=0)
data_landscape.shape

torch.Size([95567, 1024])

: 

In [ ]:
alphabet_size = 20
clustering = SpectralClustering(n_clusters=alphabet_size,
        assign_labels='discretize',
        random_state=0).fit(data_landscape)

In [ ]:
alphabet_size = 20
codebook,labels = kmeans2(data_landscape, alphabet_size, minit='points')

In [1]:
len('MKKLLVKELIEQFQDCVNLIDGHTNTSNVIRVPGLKRVVFEMLGLFSSQIGSVAILGKREFGFLSQKTLVEQQQILHNLLKLNPPAIILTKSFTDPTVLLQVNQTYQVPILKTDFFSTELSFTVETYINEQF')

132